##Note:
To get this working, we need to build and install the python wrapper for Kinfu C++. Also OpenCV source code needs to be edited to create the `getCurrentFramePoints()` method.

In [ ]:
from kinfu_cv import KinfuPlarr
import pyrealsense2 as rs
import numpy as np
import cv2

In [ ]:
focal_length = 643.338013 #942.8       # lense focal length, 1.88mm, 942.8 ???  643.338013, 643.096008
baseline = 55   #49.75  distance in mm between the two cameras
units = 0.512     # depth units, adjusted for the output to fit in one byte

width = 640
height = 480

intrinsicsNp = np.array(
    [[643.338013, 0, 638.95697 ],
     [  0, 643.096008, 402.330017],
     [  0, 0, 1.0 ]])

In [ ]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

res_x = width
res_y = height

config.enable_stream(rs.stream.depth, res_x, res_y, rs.format.z16, 15)

# Start streaming
pipeline_profile = pipeline.start(config)


## To set laser on/off or set laser power
device = pipeline_profile.get_device()
depth_sensor = device.query_sensors()[0]
laser_pwr = depth_sensor.get_option(rs.option.laser_power)
print("laser power = ", laser_pwr)
laser_range = depth_sensor.get_option_range(rs.option.laser_power)
print("laser power range = " , laser_range.min , "~", laser_range.max)
depth_sensor.set_option(rs.option.laser_power, 200)


## Get the first frame tp warm up the camera
f1 = pipeline.wait_for_frames()
d1 = f1.get_depth_frame()


#// Depth scale is needed for the kinfu_plarr set-up
depth_sensor = pipeline_profile.get_device().first_depth_sensor()

#// Set some presets for better results
preset_range = depth_sensor.get_option_range(rs.option.visual_preset)
print('preset range:'+str(preset_range))

for i in range(int(preset_range.max)):
    visulpreset = depth_sensor.get_option_value_description(rs.option.visual_preset,i)
    print(i, visulpreset)
    
    if visulpreset == "High Accuracy":
        depth_sensor.set_option(rs.option.visual_preset, i)

depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)


## define filter
spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 3)
spatial.set_option(rs.option.filter_smooth_alpha, 0.5)
spatial.set_option(rs.option.filter_smooth_delta, 20)
spatial.set_option(rs.option.holes_fill, 0)
temporal = rs.temporal_filter()
decimation = rs.decimation_filter()
# decimation.set_option(rs.option.filter_magnitude, 4)

hole_filling = rs.hole_filling_filter()
hole_filling.set_option(rs.option.holes_fill, 1)

# d1 = decimation.process(d1);
# w1 = d1.get_width();
# h1 = d1.get_height();

kfp = KinfuPlarr(width, height, depth_scale, 643.338013, 643.096008, 638.95697, 402.330017, True)


cc = 0
try:
    while True:
        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        
        if not depth_frame:
            continue

            
        # skipping first 20 frames, to wait for sensor for warm up
        cc = cc + 1
        if cc < 20:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data()) # dtype of print(depth_image.dtype) is: uint16
        
        
        ## Apply filters to real-sense depth
        filtered_rs_depth = spatial.process(depth_frame)
        filtered_rs_depth = temporal.process(filtered_rs_depth)
        filtered_rs_depth = hole_filling.process(filtered_rs_depth)
        
        #filtered_rs_depth = decimation.process(filtered_rs_depth)
        
        
        
        filtered_rs_depth = np.asanyarray(filtered_rs_depth.get_data())
        depth_image_flatten = np.array(filtered_rs_depth).flatten()
        depth_image_flatten = depth_image_flatten.tolist()
        
        isSuccess = kfp.integrateFrame(depth_image_flatten)
        if isSuccess:
            pose = kfp.getPose()
            pose = np.asarray(pose, dtype=np.float32, order='C').reshape((4, -1))
#             print(pose.shape)
#             print(pose.dtype)
            print(pose)
            
            kfp.renderShow()
        
        
            d = kfp.getCurrentDepth()
            d = np.array(d)
            d = np.nan_to_num(d)
            d = d.reshape((height, -1))
            d = d * (1/depth_scale)
            d = d.astype(np.int16)


             ## Visualize Real sense depth frames
            # Apply colormap on Real-sense depth image 
            rs_depth_sccaled  = cv2.convertScaleAbs(depth_image, alpha=0.03)  # (image converted to 8-bit per pixel)
            raycasted_depth = cv2.convertScaleAbs(d, alpha=0.03)  # (image converted to 8-bit per pixel)
            
            depth_colormap = cv2.applyColorMap(cv2.equalizeHist(rs_depth_sccaled), cv2.COLORMAP_JET)
            raycasted_depth_colormap = cv2.applyColorMap(cv2.equalizeHist(raycasted_depth), cv2.COLORMAP_JET)
            
            both_depths2 = np.hstack((depth_colormap, raycasted_depth_colormap))
            
            cv2.namedWindow('RS_DepthMap', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RS_DepthMap', both_depths2)
            key = cv2.waitKey(1)
        else:
            print("Failed")
                  
finally:

    # Stop streaming
    pipeline.stop()